In [ ]:
!pip install tensorflow==2.15

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [ ]:
!kaggle datasets download -d adityaaggarwal09/usda-barley

Dataset URL: https://www.kaggle.com/datasets/adityaaggarwal09/usda-barley
License(s): other
 83% 73.0M/87.5M [00:00<00:00, 102MB/s]
100% 87.5M/87.5M [00:00<00:00, 110MB/s]


In [ ]:



!unzip '//content/usda-barley.zip'

Streaming output truncated to the last 5000 lines.
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (31).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (32).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (33).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (34).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (35).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (36).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (37).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (38).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (39).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (4).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (40).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c1 (400).jpg  
  inflating: vitamins_detection/vitamins_detection/vitaminC/c

In [ ]:
trainPath = '/content/vitamins_detection/vitamins_detection'
testPath = '/content/test_data'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train = train_datagen.flow_from_directory(trainPath,target_size=(224,224),batch_size=15,class_mode='categorical')
test = test_datagen.flow_from_directory(testPath,target_size=(224,224),batch_size=15,class_mode='categorical')

Found 8968 images belonging to 5 classes.
Found 224 images belonging to 1 classes.


italicized text# VGG19

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
IMG_SIZE = (224,224)

In [ ]:
vgg = VGG19(include_top=False,input_shape=(224,224,3),weights='imagenet')

80134624/80134624 [==============================] - 1s 0us/step


In [ ]:
vgg.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
for layer in vgg.layers:
  print(layer)

In [ ]:
for layer in vgg.layers:
  layer.trainable=False

In [ ]:
x = Flatten()(vgg.output)

In [ ]:
output = Dense(5,activation='softmax')(x)

In [ ]:
vgg19 = Model(vgg.input,output)

In [ ]:
vgg19.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
vgg19.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

In [ ]:
# prompt: write a code to enhance picture quality for better accuracy and change it into YUV

# Install necessary libraries
!pip install opencv-python

import cv2
import numpy as np

def enhance_and_convert_to_yuv(image_path):
  # Load the image
  img = cv2.imread(image_path)

  # Enhance the image (example using histogram equalization)
  img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
  img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])
  img_enhanced = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

  return img_enhanced

# Example usage:
enhanced_image = enhance_and_convert_to_yuv('/content/vitamins_detection/vitamins_detection/train/Vitamin_B12/00000000.jpg')

# Now you can use 'enhanced_image' for further processing in your model.


In [ ]:
# prompt: write a code for edge detection for given dataset for best accuracy

import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ... (Your existing code)

def enhance_and_convert_to_yuv(image_path):
  img = cv2.imread(image_path)

  # Edge Detection using Canny (adjust thresholds as needed)
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_edges = cv2.Canny(img_gray, 100, 200)

  # Optionally, you can stack the edge information with the original image
  img_enhanced = np.dstack((img, img_edges))

  return img_enhanced

# Modify your ImageDataGenerator to use the new preprocessing function
train_datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    preprocessing_function=enhance_and_convert_to_yuv
)

# ... (Rest of your code, including model training)


In [ ]:
# prompt: write the code to train the model

history = vgg19.fit_generator(train,epochs=2,validation_data=test,steps_per_epoch=len(train),validation_steps=len(test))


<ipython-input-18-cfb0f8750a3d>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = vgg19.fit_generator(train,epochs=2,validation_data=test,steps_per_epoch=len(train),validation_steps=len(test))


Epoch 1/2
 92/598 [===>..........................] - ETA: 1:39:47 - loss: 1.6380 - accuracy: 0.4406

In [ ]:
accuracy = vgg19.evaluate(test)
print('Accuracy:', accuracy[1])

15/15 [==============================] - 1s 82ms/step - loss: 46.0443 - accuracy: 0.1116
Accuracy: 0.1116071417927742


In [ ]:
import pandas as pd
vgg19_df=pd.DataFrame(hist.history)
test_loss, test_accuracy = vgg19.evaluate(test)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)


15/15 [==============================] - 1s 81ms/step - loss: 46.0443 - accuracy: 0.1116
Test Loss: 46.044307708740234
Test Accuracy: 0.1116071417927742


In [ ]:
vgg19.save('vgg19.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from pickle import load
import numpy as np # Import the NumPy library

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
labels=['Vitamin Z','Vitamin B','Vitamin Q','Vitamin D','Vitamin E']
img_path='/content/vitamins_detection/vitamins_detection/vitaminA/00000001.jpg'
img=load_img(img_path,target_size=(224,224))
x=img_to_array(img)
x=preprocess_input(x)
# Remove the extra dimension expansion
# x=np.expand_dims(x,axis=0)
y=vgg19.predict(np.array([x])) # Now x has the correct shape
y

1/1 [==============================] - 0s 229ms/step


array([[0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
# prompt: write the code to calculate number of wrong prediction and right predictions of the model

# Assuming 'test' is your test data generator
total_predictions = 0
correct_predictions = 0
wrong_predictions = 0

for batch_index in range(len(test)):
  x_test, y_test = test.next()
  predictions = vgg19.predict(x_test)
  predicted_classes = np.argmax(predictions, axis=1)
  true_classes = np.argmax(y_test, axis=1)

  total_predictions += len(true_classes)
  correct_predictions += np.sum(predicted_classes == true_classes)
  wrong_predictions += np.sum(predicted_classes != true_classes)

print("Total Predictions:", total_predictions)
print("Correct Predictions:", correct_predictions)
print("Wrong Predictions:", wrong_predictions)


1/1 [==============================] - 0s 23ms/step
Total Predictions: 224
Correct Predictions: 25
Wrong Predictions: 199


In [ ]:
labels[np.argmax(y)]

'Vitamin E'